# Bidirectional LSTM Classifier

#### Load dependencies

In [3]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, LSTM
from keras.layers.wrappers import Bidirectional # new! 
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt 
%matplotlib inline

ImportError: No module named keras

In [5]:
#output directory name:
output_dir = 'model_output/biLSTM'

# training:
epochs = 6
batch_size = 128

# vector-space embedding: 
n_dim = 64 
n_unique_words = 10000 
max_review_length = 200 # both directions, so doubled it
pad_type = trunc_type = 'pre'
drop_embed = 0.2 

# LSTM layer architecture:
n_lstm = 256 
drop_lstm = 0.2

In [6]:
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words) # removed n_words_to_skip

NameError: name 'imdb' is not defined

In [7]:
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

NameError: name 'pad_sequences' is not defined

#### Architecture

In [9]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length)) 
model.add(SpatialDropout1D(drop_embed))
model.add(Bidirectional(LSTM(n_lstm, dropout=drop_lstm)))
model.add(Dense(1, activation='sigmoid'))

NameError: name 'Sequential' is not defined

In [10]:
model.summayr()

NameError: name 'model' is not defined

#### Configure model

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

NameError: name 'model' is not defined

In [12]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

NameError: name 'ModelCheckpoint' is not defined

In [13]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_valid, y_valid), callbacks=[modelcheckpoint])

NameError: name 'model' is not defined

In [14]:
#### Evaluate

In [15]:
model.load_weights(output_dir+"/weights.01.hdf5") # zero-indexed

NameError: name 'model' is not defined

In [16]:
y_hat = model.predict_proba(x_valid)

NameError: name 'model' is not defined

In [17]:
"{:0.2f}".format(roc_auc_score(y_valid, y_hat)*100.0)

NameError: name 'roc_auc_score' is not defined